In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [83]:
# webdriver를 이용해 kipris 접속
driver_path = r'D:\DevRoot\download\chromedriver.exe'
try: 
    driver = webdriver.Chrome(driver_path)
except(Exception):
    driver_path = r'C:\DevRoot\download\chromedriver.exe'
    driver = webdriver.Chrome(driver_path)
finally:
    driver.get("http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch")

In [84]:
# 기간을 검색어로 입력
today = datetime.today().strftime("%Y%m%d")
decade = str(int(today) - int('00100000'))
driver.find_element_by_css_selector('.keyword').send_keys(f'GD=[{decade}~{today}]')
driver.find_element_by_css_selector('.input_btn img').click()

# 90개씩 보기 선택
select = Select(driver.find_element_by_id('opt28'))
select.select_by_value('90')
driver.find_element_by_css_selector('#pageSel img').click()

In [80]:
def contentCrawling():
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    patent_list = dom.select('.search_section article')

    result = [
            {
                '제목': patent.select_one('.search_section_title h1 > a:nth-of-type(2)').text.strip(),
                '내용': patent.select_one('.search_txt').text.strip()
            }
            for patent in patent_list
    ]

    return result

In [81]:
result = []
while True:
    if len(result) == 10000: break
        
    result.extend(contentCrawling())

    # 페이징
    for i in range(10, 0, -1):
        paging = driver.find_element_by_css_selector('span.board_pager03')
        print(paging.find_element_by_css_selector(f'a:nth-last-of-type({i})').text)
        paging.find_element_by_css_selector(f'a:nth-last-of-type({i})').click()
        result.extend(contentCrawling())

2
3
4
5
6
7
8
9
10
다음
12
13
14
15
16
17
18
19
20
다음
22
23
24
25
26
27
28
29
30
다음
32
33
34
35
36
37
38
39
40
다음
42
43
44
45
46
47
48
49
50
다음
52
53
54
55
56
57
58
59
60
다음
62
63
64
65
66
67
68
69
70
다음
72
73
74
75
76
77
78
79
80


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=96.0.4664.45)


In [73]:
result

[{'제목': '확장 가능한 지압구(EXTENDABLE ACUPRESSURE DEVICE)',
  '내용': '본 고안은 지압구에 관한 것으로, 보다 상세하게는 다수의 지압구를 상호 인접되게 결합함으로써 지압 가능 구역을 확장시킬 수 있는 지압구에 관한 것이다. 본 고안에 확장 가능한 지압구는, 측면에 둘레를 따라 다수의 면을 포함하도록 다각 블럭 형태로 구성된 메인몸체와; 상기 메인몸체의 상면에 돌출 배치된 지압봉과; 인접 배치되는 다른 지압구와 자력에 의해 결합될 수 있도록 상기 메인몸체의 측면에 구비되는 결합용 자석을 포함한다.'},
 {'제목': '그물망 벤치(net type bench)',
  '내용': '본 고안은 프레임부와, 전방 지지대 및 후방 지지대와, 네트부 및 연결부를 포함하여 구성되며, 상기 네트부는 전방측에 비해 후방측이 높게 위치되도록 설치되면서 그물망으로 이루어지고, 사용자는 상기 프레임부에 걸터 앉을 수 있도록 구성된 그물망 벤치를 제공한다. 즉, 본 고안의 그물망 벤치는 단순히 앉아서 쉬는 형태의 벤치가 아니라 눕거나 기댄 상태로 휴식을 취할 수 있는 형태의 벤치이며, 특히 그물망 구조의 네트부를 제공함으로써 안락감을 더욱 향상시키면서도 안정적인 설치 상태가 유지될 수 있도록 한 것이다.'},
 {'제목': '추진장약 분리장치(SEPARATION APPARATUS FOR PROPELLING CHARGE)',
  '내용': '개시된 내용은 메쉬망에 올려진 추진장약이 좌우이동 프레임의 연속적인 좌우방향 이동과, 전후이동 프레임의 연속적인 전후방향 이동 그리고 호퍼부재에 구비되는 진동모터의 진동에 의해 상대적으로 입자가 큰 추진제와 입자가 작은 점화제로 분리되되, 추진제는 메쉬망에 걸러져 회동 프레임의 회동시 제 1 배출구를 통해 배출되고 점화제는 메쉬망을 통과해 제 2 배출구와 연결된 진공흡입식 포집기에 배출됨에 따라 신속하고 용이한 분리작업이 가능할 뿐만 아니라, 구성부재의 재료들이 전도성 기능을 갖는 스테인리스 또는 알루미늄 재질

In [82]:
len(result)

7740

In [ ]:
"""
[진행사항]
10,000 건을 크롤링하려고 했으나 7740 건에서 에러 발생
StaleElementReferenceException: Message: stale element reference: element is not attached to the page document

[특이사항]
10,000 건 크롤링 이전에 990 건을 2번 크롤링 한 것을 통해, 키프리스는 크롤링 ip 차단이 없는 것으로 보임

[처리사항]
1. 1만 4천건의 페이지를 전부 다 본다고 하면 종료시점을 어떻게 정할 것인지?
2. csv 파일 저장 코드 추가

[참고사항]
10년치 데이터를 90 건씩 14,016 pages 크롤링 -> 총 1,261,440 건
"""
None